In [2]:
import pandas as pd
import pymysql

In [3]:
url_infl = r'https://docs.google.com/spreadsheets/d/e/2PACX-1vQlI94JuTcLcdhr7rpSHzdcfcxbrf7tbtXH8BLcZm0TOGRumIMaXU76pyEzDkCly0trr_RJW5z9wcq6/pub?output=csv'
url_comp = r'https://docs.google.com/spreadsheets/d/e/2PACX-1vT30FFzpmI9eZ2bSqYhP-OnI_78d4BpU-qbHdF5fLYCuiXIjiW9i4rB2XJwJGxHQHhDapfYwhHA8hcn/pub?output=csv'

In [4]:
def get_df(url,cols,numeric_cols):
    df = pd.read_csv(url)
    rows = []
    for i in range(df.shape[0]):
        rows.append(df.iloc[i,0:len(cols)].str.split(',', expand=True))
        
    df = pd.DataFrame(pd.concat(rows).values)
    
    df.columns = cols
    for col in numeric_cols:
        df[col] = df[col].astype(np.int64)
    return df

In [5]:
cols_infl = ['instagram_id', 'country', 'followers', 'avg_likes', 'industry','price']
df_infl = get_df(url_infl, cols_infl, ['followers', 'avg_likes', 'price'])
df_infl.head()

,instagram_id,country,followers,avg_likes,industry,price
0,SEUNGRISEYO,Japan,8325402,217000,shoes,400
1,6PRI1,France,5203348,279000,shoes,300
2,YUUUKIIINAAA,Japan,4867219,100000,shoes,200
3,PIMTHA,Japan,3286925,81500,apparel,150
4,BECKYLYNCHWWE,UK,3095191,94000,apparel,200


In [6]:
cols_comp = ['name', 'country', 'followers', 'industry', 'price']
df_comp = get_df(url_comp, cols_comp, ['followers', 'price'])
df_comp.head()

,name,country,followers,industry,price
0,Converse,Japan,1000000,apparel,400
1,Zara,China,1500000,shoes,700
2,Mudo,UK,2000000,apparel,350
3,Peter England,Japan,1200000,shoes,450
4,Hackett,China,1800000,apparel,600


In [7]:
host = "firstdatabase.c3cfb1e7meof.us-east-2.rds.amazonaws.com"
port = 3306
dbname="firstdatabase"
user="agentO07"
password=""

In [8]:
conn = pymysql.connect(host=host, 
                       user=user,
                       port=port,
                       passwd=password, 
                       db=dbname,
                       cursorclass=pymysql.cursors.DictCursor)

In [9]:
# Creating INFLUENCERS table

with conn.cursor() as cursor:
    sql = "CREATE TABLE influencers (name VARCHAR(30), country VARCHAR(30), followers INT, avg_likes INT, industry VARCHAR(30), price INT);"
    cursor.execute(sql)
conn.commit()

In [10]:
# Creating COMPANIES table

with conn.cursor() as cursor:
    sql = "CREATE TABLE companies (name VARCHAR(30), country VARCHAR(30), followers INT, industry VARCHAR(30), price INT);"
    cursor.execute(sql)
conn.commit()

In [11]:
#with conn.cursor() as cursor:
    # Renaming a column
#    sql = "ALTER TABLE companies CHANGE instagram name VARCHAR(30);"
#    cursor.execute(sql)
#conn.commit()

In [12]:
# Populating INFLUENCERS table

with conn.cursor() as cursor:
    sql = "INSERT INTO influencers VALUES (%s,%s,%s,%s,%s,%s)"
    for i in range (df_infl.shape[0]):
        cursor.execute(sql,tuple(df_infl.iloc[i].apply(str).values))
conn.commit()

In [13]:
# Populating COMPANIES table

with conn.cursor() as cursor:
    sql = "INSERT INTO companies VALUES (%s,%s,%s,%s,%s)"
    for i in range (df_comp.shape[0]):
        cursor.execute(sql,tuple(df_comp.iloc[i].apply(str).values))
conn.commit()

In [14]:
#with conn.cursor() as cursor:
    # Delete a record
#    sql = "DELETE FROM influencers WHERE instagram = 'SEUNGRISEYO';"
#    cursor.execute(sql)
#conn.commit()

In [15]:
#with conn.cursor() as cursor:
    #Deleting the table
#    sql = "DROP TABLE influencers"
#    cursor.execute(sql)
#conn.commit()

In [16]:
sql = "SELECT * FROM influencers;"
pd.read_sql(sql, conn).head()

,name,country,followers,avg_likes,industry,price
0,SEUNGRISEYO,Japan,8325402,217000,shoes,400
1,6PRI1,France,5203348,279000,shoes,300
2,YUUUKIIINAAA,Japan,4867219,100000,shoes,200
3,PIMTHA,Japan,3286925,81500,apparel,150
4,BECKYLYNCHWWE,UK,3095191,94000,apparel,200


In [17]:
sql = "SELECT * FROM companies;"
pd.read_sql(sql, conn)

,name,country,followers,industry,price
0,Converse,Japan,1000000,apparel,400
1,Zara,China,1500000,shoes,700
2,Mudo,UK,2000000,apparel,350
3,Peter England,Japan,1200000,shoes,450
4,Hackett,China,1800000,apparel,600
5,Sephora,UK,2100000,shoes,800
6,GymTime,Turkey,10000,apparel,150
